# ClipCard Simulation: Industrial/Aviation
**Purpose:** This notebook runs a Monte Carlo simulation to estimate the effectiveness of ClipCard parameters for an industrial process safety scenario, such as monitoring pressure in a vessel during a temporary change (MOC).
It simulates vessel pressure over time and checks if it would trigger the `kill_criteria` defined in a ClipCard. This helps quantify the risk of a loss of containment event.

In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport json# Load example ClipCardtry:    with open('../../examples/industrial-aviation-test.clipcard.json', 'r') as f:        example_card = json.load(f)    print(f"Loaded example ClipCard: {example_card['id']}")except FileNotFoundError:    print("No example ClipCard found. Using synthetic data.")    example_card = {        "id": "SIM-INDUSTRIAL-001",        "kill_criteria": [            {"condition": "PI-V12 >= 8.5 bar for 30s", "action": "Trip FEED_PUMP_P3"}        ]    }

### Simulation Parameters
Configure simulation runs for pressure vessel monitoring.

In [ ]:
# Simulation configurationN_SIMULATIONS = 500      # Number of simulated MOC periodsN_TIMESTEPS = 12 * 60  # Number of minutes in a 12-hour shiftNORMAL_PRESSURE_BAR = 7.8 # Normal operating pressurePRESSURE_VOLATILITY = 0.05 # Standard deviation of pressure change per minuteKILL_THRESHOLD = 8.5     # From the ClipCardprint(f"Running {N_SIMULATIONS} simulations for {N_TIMESTEPS} minutes each...")print(f"Normal Pressure: {NORMAL_PRESSURE_BAR} bar")print(f"Pressure Volatility (SD per minute): {PRESSURE_VOLATILITY}")print(f"Kill Threshold: >= {KILL_THRESHOLD} bar")

### Monte Carlo Simulation
Run simulations of vessel pressure using a random walk model.

In [ ]:
results = []all_paths = []for i in range(N_SIMULATIONS):    pressure_path = [NORMAL_PRESSURE_BAR]    triggered = False    for t in range(1, N_TIMESTEPS):        # Random walk model for pressure        next_pressure = pressure_path[-1] + np.random.normal(0, PRESSURE_VOLATILITY)        if next_pressure < 0: next_pressure = 0 # Pressure can't be negative        pressure_path.append(next_pressure)        if next_pressure >= KILL_THRESHOLD:            triggered = True        all_paths.append(pressure_path)    results.append({        'sim_id': i,        'max_pressure': max(pressure_path),        'triggered': triggered    })df = pd.DataFrame(results)print(f"\nSimulation complete!")print(f"Shifts that triggered the kill criterion: {df['triggered'].sum()} ({df['triggered'].mean()*100:.1f}%)")

### Visualization
Plot simulated pressure paths and the distribution of maximum pressure values.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))# Plot a few sample pressure pathsfor i in range(min(20, N_SIMULATIONS)):    color = 'red' if max(all_paths[i]) >= KILL_THRESHOLD else 'grey'    alpha = 0.9 if color == 'red' else 0.3    axes[0].plot(all_paths[i], color=color, alpha=alpha)axes[0].axhline(KILL_THRESHOLD, color='red', linestyle='--', label=f'Kill threshold (>= {KILL_THRESHOLD} bar)')axes[0].set_xlabel('Time (minutes)')axes[0].set_ylabel('Pressure (bar)')axes[0].set_title('Sample Simulated Pressure Paths')axes[0].legend()axes[0].set_ylim(bottom=NORMAL_PRESSURE_BAR - 0.5)# Plot the distribution of maximum pressure valuesaxes[1].hist(df['max_pressure'], bins=40, alpha=0.7, edgecolor='black')axes[1].axvline(KILL_THRESHOLD, color='red', linestyle='--', label=f'Kill threshold (>= {KILL_THRESHOLD} bar)')axes[1].set_xlabel('Maximum Pressure reached during simulation (bar)')axes[1].set_ylabel('Frequency (Number of Shifts)')axes[1].set_title('Distribution of Maximum Pressure')axes[1].legend()plt.tight_layout()plt.show()print(f"\n** Interpretation **")print(f"In {N_SIMULATIONS} simulated 12-hour shifts:")print(f"- The pressure kill criterion was triggered {df['triggered'].mean()*100:.1f}% of the time.")print(f"- This helps quantify the risk of the temporary change and can inform if additional controls are needed.")